A/B testing projek

Projek ini merupakan projek/pembelajaran saya dari course yang saya ikuti pada 365datascience dan beserta dataset yang tersedia. proyek ini bertujuan untuk melakukan A/B testing yang digunakan dalam mengetahui apakah terdapat perbedaan signifikan atau dampak positif dari perubahan yang dilakukan dalam aplikasi dari segi aktivitas pengguna dan juga Click Through Rate.

Data telah dilabeli dengan 1(yaitu data test) dan 0(data kontrol), menggunakan selang kepercayaan 95% maka aplikasi A/B testing sebagai berikut

Menentukan hipotesis
H0 = Tidak ada perbedaan signifikan antara kedua grup
H1 = Terdapat perbedaan signifikan antara kedua grup

import library

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
from datetime import datetime

from scipy import stats
from scipy.stats import ttest_ind

In [ ]:
!pip install altair_viewer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 11.8 MB/s eta 0:00:00


Load dataset

TypeError: 'str' object is not callable'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AB_project/Activity_pretest.csv')
#df = pd.read_csv('/content/Activity_pretest.csv')
df.tail()

,userid,dt,activity_level
1859995,200d65e6-b1ce-4a47-8c2b-946db5c5a3a0,2021-10-31,20
1859996,535dafe4-de7c-4b56-acf6-aa94f21653bc,2021-10-31,20
1859997,0428ca3c-e666-4ef4-8588-3a2af904a123,2021-10-31,20
1859998,a8cd1579-44d4-48b3-b3d6-47ae5197dbc6,2021-10-31,20
1859999,bac5da9e-ef79-4ae9-9efe-cd6eca093db2,2021-10-31,20


In [ ]:
df.shape

(1860000, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860000 entries, 0 to 1859999
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   userid          object
 1   dt              object
 2   activity_level  int64 
dtypes: int64(1), object(2)
memory usage: 42.6+ MB


In [ ]:
df["activity_level"].value_counts().sort_values(ascending=True)

20     24520
7      48339
17     48395
8      48396
13     48534
4      48556
15     48599
14     48620
3      48659
1      48732
9      48820
11     48832
19     48901
6      48901
12     48911
16     48934
10     48943
18     48982
2      49074
5      49227
0     909125
Name: activity_level, dtype: int64

In [ ]:
aktif = df.query('activity_level > 0').groupby(['dt', 'activity_level']).count().reset_index()
aktif.head()


,dt,activity_level,userid
0,2021-10-01,1,1602
1,2021-10-01,2,1507
2,2021-10-01,3,1587
3,2021-10-01,4,1551
4,2021-10-01,5,1586


In [ ]:
#active users in oktober
alt.Chart(aktif).mark_line(size=1).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'date')),
    alt.Y('userid:Q', axis=alt.Axis(title = 'number of users')),
    tooltip=['activity_level'],
    color='activity_level:N'
).properties(
    width=600,
    height=400
)


alt.Chart(...)

In [ ]:
dailyaktif = df.query('activity_level > 0').groupby('dt')['userid'].count().reset_index(name="Total Users")
dailyaktif.head()

,dt,Total Users
0,2021-10-01,30634
1,2021-10-02,30775
2,2021-10-03,30785
3,2021-10-04,30599
4,2021-10-05,30588


In [ ]:
alt.Chart(dailyaktif).mark_line(size=6).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'date')),
    alt.Y('Total Users:Q', axis=alt.Axis(title = 'number of users')),
    tooltip=['Total Users'],
    color=alt.value('red')
).properties(
    width=600,
    height=400
)


alt.Chart(...)

CTR

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/AB_project/Ctr_pretest.csv')
df1.head()

,userid,dt,ctr
0,4b328144-df4b-47b1-a804-09834942dce0,2021-10-01,34.28
1,34ace777-5e9d-40b3-a859-4145d0c35c8d,2021-10-01,34.67
2,8028cccf-19c3-4c0e-b5b2-e707e15d2d83,2021-10-01,34.77
3,652b3c9c-5e29-4bf0-9373-924687b1567e,2021-10-01,35.42
4,45b57434-4666-4b57-9798-35489dc1092a,2021-10-01,35.04


In [ ]:
ctr = df1.groupby(['dt']).mean().reset_index()

In [ ]:
alt.Chart(ctr).mark_line(size=4).encode(
    alt.X('dt:T', axis=alt.Axis(title = 'date')),
    alt.Y('ctr:Q', axis=alt.Axis(title = 'ctr'), scale=alt.Scale(domain=[32, 34])),
    tooltip=['ctr'],
    color=alt.value('red')
).properties(
    width=600,
    height=400,
    title='Average Daily CTR'
)

alt.Chart(...)

Significance Power

In [ ]:
def jumlah_sample_binomial(metric, mde, alpha, beta):
    # standard normal distribution to determine z-values
    snd = stats.norm(0, 1)

    Z_beta = snd.ppf(1-beta)
    print("Nilai beta", Z_beta)

    Z_alpha = snd.ppf(1-alpha/2)
    print("Nilai Alpha", Z_alpha)

    # average of probabilities from both groups
    p = (metric + metric+mde) / 2
    print("Nilai Percentage Point", p)
    #sample size
    N = (2 * p *
             (1 - p) *
             (0.84 + 1.96)**2
             / mde**2)

    return N

In [ ]:
jumlah_sample_binomial(metric=0.33, mde=0.02, alpha=0.05, beta=0.2)

Nilai beta 0.8416212335729143
Nilai Alpha 1.959963984540054
Nilai Percentage Point 0.34


8796.479999999998

In [ ]:
def jumlah_sample_continous(metric, mde, std, alpha, beta):
    # standard normal distribution to determine z-values
    snd = stats.norm(0, 1)

    Z_beta = snd.ppf(1-beta)
    print("Nilai beta", Z_beta)


    Z_alpha = snd.ppf(1-alpha/2)
    print("Nilai Alpha", Z_alpha)

    N = (2 * std**2 *
             (Z_beta + Z_alpha)**2
             / mde**2)

    return N

In [ ]:
jumlah_sample_continous(metric=30673, mde=300, std=91, alpha=0.05, beta=0.2)

Nilai beta 0.8416212335729143
Nilai Alpha 1.959963984540054


1.4443682906698845

Hipotesis test

Load data activit

In [ ]:
dfactivity = pd.read_csv('/content/drive/MyDrive/AB_project/Activity_all.csv')
dfactivity.head()


,userid,dt,groupid,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,1,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,1,0


In [ ]:
dfactivity.groupby(["groupid", "dt"])["userid"].count()

groupid  dt        
0        2021-10-01    29951
         2021-10-02    29951
         2021-10-03    29951
         2021-10-04    29951
         2021-10-05    29951
                       ...  
1        2021-11-26    30049
         2021-11-27    30049
         2021-11-28    30049
         2021-11-29    30049
         2021-11-30    30049
Name: userid, Length: 122, dtype: int64

In [ ]:
dfactivity["activity_level"].value_counts()

0     1356592
2      119053
1      118649
12     118549
9      118472
19     118333
18     118305
16     118298
11     118246
5      118184
15     118148
6      118099
3      118087
4      117892
13     117861
17     117820
14     117695
10     117669
8      117613
7      117359
20      59076
Name: activity_level, dtype: int64

In [ ]:
dfactivitygrup = dfactivity.query('activity_level > 0').groupby(['dt', 'groupid']).count().reset_index()
dfactivitygrup.head()

,dt,groupid,userid,activity_level
0,2021-10-01,0,15337,15337
1,2021-10-01,1,15297,15297
2,2021-10-02,0,15354,15354
3,2021-10-02,1,15421,15421
4,2021-10-03,0,15423,15423


In [ ]:
alt.Chart(dfactivity.query('activity_level > 0').groupby(['dt', 'groupid'])["userid"].count().reset_index()).mark_line(size=3).encode(
    alt.X('dt'),
    alt.Y('userid'),
    #color='groupid:O',
    color=alt.Color('groupid:O', scale=alt.Scale(range=['orange']), legend=alt.Legend(title='Group ID')),
    opacity=alt.condition(
        alt.datum.groupid == 0,  # Set opasitas berdasarkan kondisi groupid
        alt.value(0.5),  # Jika groupid == 1, opasitas 0.5
        alt.value(0.9)   # Jika groupid == 0, opasitas 0.8
    ),
    tooltip=['userid']

).properties(
    width=600,
    height=400
)



alt.Chart(...)

pada grup test yaitu yang dilabeli 1 ternyata memiliki peningkatan sejak tanggal 2021-11-01, selanjutnya akan diselidiki apakah terdapat perbedaan CTR signifikan antara grup test(1) dan grup kontrol(0)

In [ ]:

dfactivity.query('activity_level > 0 and groupid == 0 and dt >= "2021-11-01"').groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe()


,groupid,activity_level
count,30.0,30.000000
mean,0.0,15782.000000
std,0.0,371.077276
min,0.0,15163.000000
25%,0.0,15335.000000
50%,0.0,15990.500000
75%,0.0,16045.000000
max,0.0,16147.000000


In [ ]:
dfactivity.query('activity_level > 0 and groupid == 1 and dt >= "2021-11-01"').groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe()

,groupid,activity_level
count,30.0,30.000000
mean,1.0,29302.433333
std,0.0,30.417422
min,1.0,29255.000000
25%,1.0,29280.000000
50%,1.0,29300.000000
75%,1.0,29321.000000
max,1.0,29382.000000


In [ ]:
dfactivity.query('activity_level > 0 and groupid == 1 and dt >= "2021-11-01"').groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe()

,groupid,activity_level
count,30.0,30.000000
mean,1.0,29302.433333
std,0.0,30.417422
min,1.0,29255.000000
25%,1.0,29280.000000
50%,1.0,29300.000000
75%,1.0,29321.000000
max,1.0,29382.000000


Compare activity between groups

In [ ]:
result = ttest_ind(
    dfactivity.query('groupid == 0 and dt >= "2021-11-01"')['activity_level'].to_numpy(),
    dfactivity.query('groupid == 1 and dt >= "2021-11-01"')['activity_level'].to_numpy()
)

p_value = result.pvalue

print("Nilai p-value:", p_value)

# Interpretasi hasil
if p_value < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.0
Terdapat perbedaan signifikan antara dua kelompok.


Karena terdapat perbedaan signifikan antara dua kelompok grup test dan juga kontrol maka dilakukan bias pretest Bias pretest terjadi ketika ada perbedaan mendasar antara kelompok kontrol dan kelompok perlakuan sebelum adanya intervensi.


In [ ]:
before = dfactivitygrup.query('dt < "2021-11-01"')
after = dfactivitygrup.query('dt >= "2021-11-01"')

In [ ]:
result2 = ttest_ind(before.query('groupid == 0')['userid'].to_numpy(), before.query('groupid == 1')['userid']
                .to_numpy()).pvalue

print("Nilai p-value:", result2)

# Interpretasi hasil
if result2 < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.1630842353828084
Tidak terdapat perbedaan signifikan antara dua kelompok.


ternyata sebelum intervensi dilakukan antara 2 kelompok grup ini tidak memiliki perbedaan signifikan

In [ ]:
result3 = ttest_ind(after.query('groupid == 0')['userid'].to_numpy(), after.query('groupid == 1')['userid']
                .to_numpy()).pvalue

print("Nilai p-value:", "{:.100f}".format(result3))

# Interpretasi hasil
if result3 < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000065906035841072442
Terdapat perbedaan signifikan antara dua kelompok.


Dan setelah intervensi memiliki perbedaan signifikan, artinya pada hasil sebelum bias pretest dapat dipastikan bahwa Adanya perbedaan signifikan (alpha 0.05) antara grup kontrol dan test itu berasal dari hasil intervensi bukan berbeda signifikan sejak awal sebelum diintervensi.

Kesimpulan : Dengan melakukan intervensi atau perubahan sejak tanggal 2021-11-01 hal tersebut mampu meningkatkan aktivitas pengguna secara signifikan

Load data ctr

In [ ]:
dfctr = pd.read_csv('/content/drive/MyDrive/AB_project/Ctr_all.csv')
dfctr.head()

,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95


In [ ]:
dfctraverage = dfctr.groupby(['groupid','dt']).mean().reset_index()
dfctraverage.head()

,groupid,dt,ctr
0,0,2021-10-01,32.980627
1,0,2021-10-02,33.004056
2,0,2021-10-03,33.002006
3,0,2021-10-04,32.990363
4,0,2021-10-05,33.014167


In [ ]:
alt.Chart(dfctraverage).mark_line(size=4).encode(
    alt.X('dt'),
    alt.Y('ctr'),
    color='groupid:O',
    tooltip=['ctr']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

pada grup test yaitu yang dilabeli 1 ternyata memiliki peningkatan sejak tanggal 2021-11-01, selanjutnya akan diselidiki apakah terdapat perbedaan CTR signifikan antara grup test(1) dan grup kontrol(0)

Comparing CTR between groups

In [ ]:
result4 = ttest_ind(
    dfctr.query('groupid == 0 and dt >= "2021-11-01"')['ctr'].to_numpy(),
    dfctr.query('groupid == 1 and dt >= "2021-11-01"')['ctr'].to_numpy()).pvalue


print("Nilai p-value:", result4)

# Interpretasi hasil
if result4 < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.0
Terdapat perbedaan signifikan antara dua kelompok.


Kita akan melakukan bias pretest kembali apakah data sebelum intervensi itu sudah berbeda signifikan sebelumnya

In [ ]:
before1 = dfctr.query('dt < "2021-11-01"')[['groupid', 'ctr']]
after1 = dfctr.query('dt >= "2021-11-01"')[['groupid', 'ctr']]

In [ ]:
after1.query('groupid == 0')['ctr'].to_numpy().std()

1.7331985918552912

In [ ]:
kondisi = [before1,after1]
grupnya = [0,1]

for i in kondisi :
  for j in grupnya :
      std_dev = i.query('groupid == @j')['ctr'].to_numpy().std()
      print(f"Standar deviasi untuk grup {j} pada kondisi adalah: {std_dev}")

Standar deviasi untuk grup 0 pada kondisi adalah: 1.7336979501682888
Standar deviasi untuk grup 1 pada kondisi adalah: 1.7296548367391134
Standar deviasi untuk grup 0 pada kondisi adalah: 1.7331985918552912
Standar deviasi untuk grup 1 pada kondisi adalah: 1.7323710606903675


In [ ]:
result5 = ttest_ind(before1.query('groupid == 0')['ctr'].to_numpy(), before1.query('groupid == 1')['ctr']
                .to_numpy()).pvalue

print("Nilai p-value:", result5)

# Interpretasi hasil
if result5 < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.705741417344299
Tidak terdapat perbedaan signifikan antara dua kelompok.


Sebelum dilakukan intervensi yaitu sebelum tanggal 2021-11-01, tidak ada perbedaan signifikan antara data kontrol dan data test

In [ ]:
result6 = ttest_ind(after1.query('groupid == 0')['ctr'].to_numpy(), after1.query('groupid == 1')['ctr']
                .to_numpy()).pvalue

print("Nilai p-value:", result6)

# Interpretasi hasil
if result6 < 0.05:
    print("Terdapat perbedaan signifikan antara dua kelompok.")
else:
    print("Tidak terdapat perbedaan signifikan antara dua kelompok.")

Nilai p-value: 0.0
Terdapat perbedaan signifikan antara dua kelompok.


Dan setelah intervensi memiliki perbedaan signifikan, artinya pada hasil sebelum bias pretest dapat dipastikan bahwa Adanya perbedaan signifikan (alpha 0.05) antara grup kontrol dan test itu berasal dari hasil intervensi bukan berbeda signifikan sejak awal sebelum diintervensi.

Kesimpulan : Dengan melakukan intervensi atau perubahan sejak tanggal 2021-11-01 hal tersebut mampu meningkatkan CTR pengguna secara signifikan

Bussines recommendation : perubahan/intervensi yang dilakukan sejak tanggal 2021-11-01 menimbulkan dampak positif terhadap aplikasi yaitu meningkatkan aktivitas pengguna dan juga Click Through Rate (CTR) ikut meningkat. Oleh karena itu keputusan perusahaan dalam melakukan intervensi sangat baik dan dapat dilakukan proses selanjutnya yaitu menerapkan strategi intervensi ke seluruh pengguna/populasi. Namun harus dilakukan analisis lebih mendalam kembali terkait faktor-faktor yang lain